## QBUS3850 Lab 4 (ARIMA Models) Task

In this tutorial we will work on the Minneapolis traffic data available on Canvas. This data contains daily observations of traffic volume and was originally sources from John Hogue who made it available at the [UCI machine learning repository](https://archive.ics.uci.edu/ml/datasets/Metro+Interstate+Traffic+Volume). Some post-process of the data has been carried out.

Your task will be to use all data up to 2018-07-31 to forecast the next two months ahead (August and September 2018). You will use the following models

- An AR(1)
- An MA(1)
- An ARIMA selected by auto_arima
- A seasonal ARIMA selected by auto_arima 
- A regression with holiday dummies and errors following seasonal ARIMA selected by auto_arima 
- A regression with holiday dummies and Fourier terms and errors following seasonal ARIMA selected by auto_arima

After estimating each model, plot the forecasts together with some training data and the test data. Comment on how the forecasts reflect the properties of each model.

### Autoregressive Model (AR1)